In [23]:
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
import openai
client = openai.OpenAI()
openai.api_key = OPENAI_API_KEY

In [18]:
from pydantic import BaseModel, Field
from typing import Optional, List

class IdentificationDetails(BaseModel):
    legal_id: Optional[str] = Field(None, description="Type of Legal ID (e.g., National Identity Card)")
    number_type: str = Field(..., description="Number or type of the ID")
    issuing_office: Optional[str] = Field(None, description="Issuing office of the ID")
    date_of_issue: str = Field(..., description="Date of issue (in YYYY-MM-DD format)")
    expiry_date: Optional[str] = Field(None, description="Expiry date (in YYYY-MM-DD format)")
    primary_secondary: Optional[str] = Field(None, description="Primary/Secondary designation")

class RelatedParty(BaseModel):
    relation: Optional[str] = Field(None, description="Relation (e.g., Father, Mother)")
    full_name: str = Field(..., description="Full name of the related party")
    id_type: Optional[str] = Field(None, description="Type of ID for the related party")
    id_no: Optional[str] = Field(None, description="ID number of the related party")
    have_account: Optional[str] = Field(None, description="Whether the related party has an account (Yes/No)")
    customer_no: Optional[str] = Field(None, description="Customer number if applicable")

class PersonalAccountForm(BaseModel):
    date: Optional[str] = Field(None, description="Form submission date (in YYYY-MM-DD format)")
    branch: Optional[str] = Field(None, description="Branch name")
    account_type: str = Field(..., description="Type of account (e.g., Saving, Current)")
    product_name: Optional[str] = Field(None, description="Name of the product, if applicable")
    currency: str = Field(..., description="Selected currency (e.g., NPR, USD)")
    salutation: Optional[str] = Field(None, description="Salutation (e.g., Mr., Ms.)")
    full_name: str = Field(..., description="Full name of the account holder")
    date_of_birth_bs: str = Field(..., description="Date of birth in B.S. format")
    date_of_birth_ad: str = Field(..., description="Date of birth in A.D. format")
    gender: str = Field(..., description="Gender (e.g., Male, Female, Others)")
    marital_status: str = Field(..., description="Marital status (e.g., Married, Unmarried)")
    nationality: str = Field(..., description="Nationality of the account holder")
    resident: Optional[str] = Field(None, description="Resident status, if applicable")
    education: Optional[str] = Field(None, description="Education level (e.g., Literate, Illiterate)")
    existing_account: str = Field(..., description="Whether the account holder has an existing account (Yes/No)")
    account_no: Optional[str] = Field(None, description="Existing account number, if applicable")
    identification_details: List[IdentificationDetails] = Field(..., description="List of identification details")
    related_parties: Optional[List[RelatedParty]] = Field(None, description="List of related parties")


In [34]:
system_prompt = f"""
    You are an OCR-like data extraction tool that extracts personal information from the image provided of a form for opening a bank account.
   
    1. Please extract the data in this image, grouping data according to theme/sub groups, and then output into JSON.

    2. Please keep the keys and values of the JSON in the original language. 

    3. The type of data you might encounter in the image includes but is not limited to: names, dates, checkboxes, nepali language    
    
    4. If there are tables in the image, capture all of the rows and columns in the JSON object. 
    Even if a column is blank, include it as a key in the JSON object with a null value.
    
    5. Don't interpolate or make up data.

    6. Please maintain the table structure of the charges, i.e. capture all of the rows and columns in the JSON object.

    7. Return null if the data is not available

    8. If no checkboxes are selected, just return null.

    9. Triple check any numbers provided in the attatched image.

    10. Properly check which row the data belongs to.

    """

In [35]:
import base64

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "D:\Programming\AI\Basics\AMNIL Tech\OCR\ocrimage.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role":"system",
            "content":system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Extract the information in this bank account opening form and output into JSON.",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        }
    ],
    response_format=PersonalAccountForm,
)

In [37]:
response.choices[0].message.content

'{\n  "date": "2025-02-17",\n  "branch": null,\n  "account_type": "Saving Account",\n  "product_name": null,\n  "currency": "NPR",\n  "salutation": "Rakhi",\n  "full_name": "Mahato",\n  "date_of_birth_bs": "2057-02-05",\n  "date_of_birth_ad": "2000-02-17",\n  "gender": "Female",\n  "marital_status": "Unmarried",\n  "nationality": "Nepalese",\n  "resident": "Resident",\n  "education": "Literate",\n  "existing_account": "No",\n  "account_no": null,\n  "identification_details": [\n    {\n      "legal_id": "National identity card",\n      "number_type": "17-02-75-0027",\n      "issuing_office": "2075102116",\n      "date_of_issue": "2075-02-17",\n      "expiry_date": null,\n      "primary_secondary": "Primary"\n    }\n  ],\n  "related_parties": [\n    {\n      "relation": "Spouse",\n      "full_name": "Sita Devi Mahato",\n      "id_type": null,\n      "id_no": null,\n      "have_account": "No",\n      "customer_no": null\n    },\n    {\n      "relation": "Mother",\n      "full_name": "Bhuw

In [46]:
content = response.choices[0].message.content

In [39]:
import json
parsed_response = json.loads(response.choices[0].message.content)


pretty_json = json.dumps(parsed_response, indent=4)

In [50]:
json.loads(content)

{'date': '2025-02-17',
 'branch': None,
 'account_type': 'Saving Account',
 'product_name': None,
 'currency': 'NPR',
 'salutation': 'Rakhi',
 'full_name': 'Mahato',
 'date_of_birth_bs': '2057-02-05',
 'date_of_birth_ad': '2000-02-17',
 'gender': 'Female',
 'marital_status': 'Unmarried',
 'nationality': 'Nepalese',
 'resident': 'Resident',
 'education': 'Literate',
 'existing_account': 'No',
 'account_no': None,
 'identification_details': [{'legal_id': 'National identity card',
   'number_type': '17-02-75-0027',
   'issuing_office': '2075102116',
   'date_of_issue': '2075-02-17',
   'expiry_date': None,
   'primary_secondary': 'Primary'}],
 'related_parties': [{'relation': 'Spouse',
   'full_name': 'Sita Devi Mahato',
   'id_type': None,
   'id_no': None,
   'have_account': 'No',
   'customer_no': None},
  {'relation': 'Mother',
   'full_name': 'Bhuwaneshwor Prasad Mahato',
   'id_type': None,
   'id_no': None,
   'have_account': 'No',
   'customer_no': None},
  {'relation': 'Grandfath